In [1]:
# HIDDEN - sets path

import sys
sys.path.insert(1, '../..')

# Managing asset access
Access to assets is managed in ContextHub rather than ConfigHub, and permissions can only be set per user, not per group. To get around this we can load the group members in ConfigHub, and loop over them to give them access in the appliance.

## Creating clients
We need client to both the appliance and ConfigHub. The appliance authentication needs a user with admin rights.

In [2]:
import keyring
from trendminer_interface import TrendMinerClient
from confighub_interface import ConfigClient


url = "https://cs.trendminer.net"
client_id = "wdanielsclient"
username = "wdaniels"

client = TrendMinerClient(
    url=url, 
    client_id=client_id, 
    client_secret=keyring.get_password(url, client_id),
    username=username,
    password=keyring.get_password(url, username)
)

cfg = ConfigClient(
    url=url, 
    password=keyring.get_password(url, "admin")
)

In [3]:
# HIDDEN
new_user = cfg.user(
        name="MyUser",
        first="My",
        last="User",
        mail="MyUser@trendminer.com"
    )
new_user.post("MyPassword1!")


group = cfg.group("MyGroup")
group.post()
group.member_add(new_user)

## Loading the asset on which to set permissions

In [4]:
asset = client.asset.from_path("AF-example")

## Loading users from group
First, let's extract the users from a group. Since a group can contain another group, we can write a recursive function to extract the members from subgroups too.

In [5]:
members = []

def extract_members(group):
    for member in group.members_get():
        if isinstance(member, cfg.user.tm_class):
            members.append(member)
        else:
            extract_members(member)  # member is a subgroup
            
extract_members(cfg.group.from_name("MyGroup"))
members

[<< ConfigUser | myuser >>]

The retrieved users are user instances specific to ConfigHub. Users in the appliance are similar, but are represented by a separate class. Since setting access to the asset happens in the appliance, we need to retrieve the corresponding appliance users from the ConfigHub users.

In [6]:
users = [client.user.from_name(member.name) for member in members]
users

[<< User | myuser >>]

## Setting permissions
The permissions are valid for the asset and all its children.

In [7]:
# Add access for all users (options: none, browse, read)
for user in users:
    asset.access.add(user, "read") # read access includes browse access

# We can confirm access has been given
print(asset.access.all())

[<< AccessRule | myuser: ASSET_BROWSE,ASSET_READ_CONTEXT_ITEM >>]


## Clearing permissions
There is no method yet for clearing access of one particular user, but we can revoke all access at once.

In [8]:
asset.access.clear()

In [9]:
# HIDDEN
new_user.delete()
group.delete()